In [1]:
param_date = '20240715'

In [2]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [3]:
input_ = pd.read_excel(f'./{param_date}/2.group.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Manufacturer No'] = input_['Manufacturer No'].astype(int)
input_['Group No'] = input_['Group No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Group'].tolist()

总数量：2525


['Body & Lamp Assembly',
 'Interior',
 'Body & Lamp Assembly',
 'Engine',
 'Garage Equipment',
 'Interior',
 'Steering',
 'Interior',
 'Fuel & Air',
 'Ignition',
 'Belt Drive',
 'Body & Lamp Assembly',
 'Brake & Wheel Hub',
 'Cooling System',
 'Drivetrain',
 'Electrical',
 'Electrical-Bulb & Socket',
 'Electrical-Connector',
 'Electrical-Switch & Relay',
 'Engine',
 'Exhaust & Emission',
 'Fuel & Air',
 'Garage Equipment',
 'Hardware',
 'Heat & Air Conditioning',
 'Hoses/Lines & Clamps',
 'Ignition',
 'Interior',
 'Steering',
 'Suspension',
 'Transmission-Automatic',
 'Transmission-Manual',
 'Wheel',
 'Wiper & Washer',
 'Ignition',
 'Body & Lamp Assembly',
 'Electrical-Connector',
 'Electrical-Switch & Relay',
 'Garage Equipment',
 'Heat & Air Conditioning',
 'Interior',
 'Wiper & Washer',
 'Body & Lamp Assembly',
 'Interior',
 'Ignition',
 'Brake & Wheel Hub',
 'Fuel & Air',
 'Cooling System',
 'Engine',
 'Heat & Air Conditioning',
 'Wiper & Washer',
 'Brake & Wheel Hub',
 'Electrical

In [4]:
a = 0

a

0

In [5]:
crawler_status = 'error'

crawler_status

'error'

In [6]:
import requests
import json

import sys
sys.path.append('../../00.Tools')
from crawler_configuration import get_header, get_proxy

data = {'func': 'partsearchgetparttypessdd',
        'mfr': input_.loc[a, 'Manufacturer'],
        'pg': input_.loc[a, 'Group'],
        'giveback': json.dumps({'id': 'partsearch_007'}),
        'api_json_request': '1'}

b = 0
while True:

    while True:
        b += 1
        print(b)
        try:
            resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                 data=data,
                                 headers=get_header(),
                                 proxies=get_proxy(),
                                 timeout=(10, 10))
            break
        except:
            continue

    if resp.status_code == 200:
        break

resp

1


<Response [200]>

In [7]:
json_ = resp.json()

json_

{'html_replace_sections': {'ddrepl[parttype_][container]': '<span id="ddrepl[parttype_][container]" class="ra-vertical-dropdown-replacement-c"><select id="parttype_" name="partsearch[parttype][]" onchange = "" tabindex="0" autocomplete = "none"><option value=""  selected >All</option><option value="1263"  >Cargo Bag</option><option value="1000410"  >Cargo Carrier</option><option value="1104"  >Roof Rack</option><option value="1152"  >Sunroof Wind Deflector</option></select></span>'},
 'giveback': {'id': 'partsearch_007'},
 'collected_javascript': '/**UQ:1F7A8A93**/\nwindow.top.parent.window["_nck"] = "8LVLUxTaILBT+tLxkg8+3Ag7\\/0bbOhvn7P3rkFbL809YNOHQDmNBznFtdYfbcEUriQ0mcmy\\/TcWfcbfHTjxSwndfe2chgzy88liHO2MoAcp7wW7lek5DxawTLSg\\/DpYUwwgKwrMfy32BI6R68c\\/w7j6KldZsshRcNdDKSl4lF2c=";\n'}

In [ ]:
'redirect_to_url' in json_

In [8]:
json_['giveback']['id'] == 'partsearch_007' and 'collected_javascript' in json_

True

In [9]:
from bs4 import BeautifulSoup
from lxml import etree

soup = BeautifulSoup(json_['html_replace_sections']['ddrepl[parttype_][container]'], 'lxml')
html = etree.HTML(str(soup))

with open('./type.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<html>
 <body>
  <span class="ra-vertical-dropdown-replacement-c" id="ddrepl[parttype_][container]">
   <select autocomplete="none" id="parttype_" name="partsearch[parttype][]" onchange="" tabindex="0">
    <option selected="" value="">
     All
    </option>
    <option value="1263">
     Cargo Bag
    </option>
    <option value="1000410">
     Cargo Carrier
    </option>
    <option value="1104">
     Roof Rack
    </option>
    <option value="1152">
     Sunroof Wind Deflector
    </option>
   </select>
  </span>
 </body>
</html>



In [10]:
list_type_code = html.xpath('//select[@id="parttype_"]/option/@value')[1:]

list_type_code

['1263', '1000410', '1104', '1152']

In [11]:
list_type = html.xpath('//select[@id="parttype_"]/option/text()')[1:]

list_type

['Cargo Bag', 'Cargo Carrier', 'Roof Rack', 'Sunroof Wind Deflector']

In [12]:
len(list_type) == 0

False

In [13]:
df_temp = pd.DataFrame({'Manufacturer No': input_.loc[a, 'Manufacturer No'],
                        'Manufacturer': input_.loc[a, 'Manufacturer'],
                        'Group No': input_.loc[a, 'Group No'],
                        'Group': input_.loc[a, 'Group'],
                        'Type No': [i+1 for i in range(len(list_type))],
                        'Type': list_type,
                        'Type Code': list_type_code,
                        'Tab 2 Url': [f'''https://www.rockauto.com/en/parts/{input_.loc[a, 'Manufacturer'].replace(',', '%2C').replace('+', '%2B')},{type_.replace(',', '%2C')},{type_code.replace(',', '%2C')}''' for type_, type_code in zip(list_type, list_type_code)]})

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,Manufacturer No,Manufacturer,Group No,Group,Type No,Type,Type Code,Tab 2 Url
0,1,3D,1,Body & Lamp Assembly,1,Cargo Bag,1263,"https://www.rockauto.com/en/parts/3D,Cargo Bag..."
1,1,3D,1,Body & Lamp Assembly,2,Cargo Carrier,1000410,"https://www.rockauto.com/en/parts/3D,Cargo Car..."
2,1,3D,1,Body & Lamp Assembly,3,Roof Rack,1104,"https://www.rockauto.com/en/parts/3D,Roof Rack..."
3,1,3D,1,Body & Lamp Assembly,4,Sunroof Wind Deflector,1152,"https://www.rockauto.com/en/parts/3D,Sunroof W..."


In [14]:
crawler_status = 'ok'

crawler_status

'ok'

In [15]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([{'Manufacturer No': input_.loc[a, 'Manufacturer No'],
                             'Manufacturer': input_.loc[a, 'Manufacturer'],
                             'Group No': input_.loc[a, 'Group No'],
                             'Group': input_.loc[a, 'Group']}])

    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

output_error

""


In [16]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Manufacturer No', 'Group No', 'Type No'],
                                                ascending=[True, True, True],
                                                ignore_index=True)
    output_correct.to_excel('./test_type.xlsx', index=False)

output_correct

,Manufacturer No,Manufacturer,Group No,Group,Type No,Type,Type Code,Tab 2 Url
0,1,3D,1,Body & Lamp Assembly,1,Cargo Bag,1263,"https://www.rockauto.com/en/parts/3D,Cargo Bag..."
1,1,3D,1,Body & Lamp Assembly,2,Cargo Carrier,1000410,"https://www.rockauto.com/en/parts/3D,Cargo Car..."
2,1,3D,1,Body & Lamp Assembly,3,Roof Rack,1104,"https://www.rockauto.com/en/parts/3D,Roof Rack..."
3,1,3D,1,Body & Lamp Assembly,4,Sunroof Wind Deflector,1152,"https://www.rockauto.com/en/parts/3D,Sunroof W..."


In [17]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['Manufacturer No', 'Group No'],
                                            ascending=[True, True],
                                            ignore_index=True)
    output_error.to_excel('./test_type_error.xlsx', index=False)

output_error

""
